# 🎬 Netflix Movie Recommender with Posters
This notebook builds a movie recommender system using metadata, ratings, and TMDB API posters.

In [ ]:

import os
import pandas as pd
import numpy as np
import pickle
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Image, display


In [ ]:

# Load datasets (ensure CSVs are in working directory or use Kaggle API)
movies = pd.read_csv("movies_metadata.csv", low_memory=False)
ratings = pd.read_csv("ratings_small.csv")
credits = pd.read_csv("credits.csv")
keywords = pd.read_csv("keywords.csv")
links = pd.read_csv("links_small.csv")

print("Movies:", movies.shape)
print("Ratings:", ratings.shape)
print("Credits:", credits.shape)
print("Keywords:", keywords.shape)
print("Links:", links.shape)


In [ ]:

# Use 'overview' text for TF-IDF similarity
movies['overview'] = movies['overview'].fillna('')

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['overview'])

similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
print("Similarity matrix shape:", similarity.shape)


In [ ]:

# Save similarity matrix to pickle
with open("similarity.pkl", "wb") as f:
    pickle.dump(similarity, f)
print("✅ similarity.pkl saved.")


In [ ]:

# --- TMDB API ---
TMDB_API_KEY = "0d309fbe7061ac46435369d2349288ba"   # Replace with your own key

def fetch_poster(movie_title):
    url = f"https://api.themoviedb.org/3/search/movie?api_key={TMDB_API_KEY}&query={movie_title}"
    response = requests.get(url).json()
    if response.get("results"):
        poster_path = response["results"][0].get("poster_path")
        if poster_path:
            return "https://image.tmdb.org/t/p/w500" + poster_path
    return None

# Test poster fetch
print(fetch_poster("Toy Story"))


In [ ]:

def recommend(movie, n=5):
    matches = movies[movies['original_title'].str.lower() == movie.lower()]
    if matches.empty:
        return []
    idx = matches.index[0]
    distances = list(enumerate(similarity[idx]))
    distances = sorted(distances, key=lambda x: x[1], reverse=True)[1:n+1]
    rec_idxs = [i for i, _ in distances]
    
    rec_movies = movies.loc[rec_idxs, ["original_title", "release_date", "vote_average"]].copy()
    rec_movies["poster"] = rec_movies["original_title"].apply(fetch_poster)
    return rec_movies


In [ ]:

# Example usage
recs = recommend("Toy Story", 5)
recs


In [ ]:

# Show posters inline
for _, row in recs.iterrows():
    display(Image(url=row["poster"], width=150))
    print(f"{row['original_title']} ({row['release_date'][:4] if pd.notnull(row['release_date']) else 'N/A'}) ⭐ {row['vote_average']}")
